# Get Dataset from Google Drive
please upload your dataset on google drive first.

In [7]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
import os
import zipfile
import tqdm

file_name = "Multimedia_dataset.zip"
zip_path = os.path.join('/content/drive/MyDrive/Multimedia_dataset.zip')

!cp "{zip_path}" .
!unzip -q "{file_name}"
!rm "{file_name}"

# Color Hint Transform
If you want to change how many hints you are giving, change the threshold values in call function.

In [8]:
import torch
from torch.autograd import Variable
from torchvision import transforms

import cv2
import random
import numpy as np

class ColorHintTransform(object):
  def __init__(self, size=256, mode="training"):
    super(ColorHintTransform, self).__init__()
    self.size = size
    self.mode = mode
    self.transform = transforms.Compose([transforms.ToTensor()])

  def bgr_to_lab(self, img):
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, ab = lab[:, :, 0], lab[:, :, 1:]
    return l, ab

  def hint_mask(self, bgr, threshold=[0.95, 0.97, 0.99]):
    h, w, c = bgr.shape
    mask_threshold = random.choice(threshold)
    mask = np.random.random([h, w, 1]) > mask_threshold
    return mask

  def img_to_mask(self, mask_img):
    mask = mask_img[:, :, 0, np.newaxis] >= 255
    return mask

  def __call__(self, img, mask_img=None):
    threshold = [0.95, 0.97, 0.99]
    if (self.mode == "training") | (self.mode == "validation"):
      image = cv2.resize(img, (self.size, self.size))
      mask = self.hint_mask(image, threshold)

      hint_image = image * mask

      l, ab = self.bgr_to_lab(image)
      l_hint, ab_hint = self.bgr_to_lab(hint_image)

      return self.transform(l), self.transform(ab), self.transform(ab_hint)

    elif self.mode == "testing":
      image = cv2.resize(img, (self.size, self.size))
      hint_image = image * self.img_to_mask(mask_img)

      l, _ = self.bgr_to_lab(image)
      _, ab_hint = self.bgr_to_lab(hint_image)

      return self.transform(l), self.transform(ab_hint)

    else:
      return NotImplementedError

# Dataloader for Colorization Dataset

In [9]:
import torch
import torch.utils.data  as data
import os
import cv2
from google.colab.patches import cv2_imshow

class ColorHintDataset(data.Dataset):
  def __init__(self, root_path, size):
    super(ColorHintDataset, self).__init__()

    self.root_path = root_path
    self.size = size
    self.transforms = None
    self.examples = None
    self.hint = None
    self.mask = None

  def set_mode(self, mode):
    self.mode = mode
    self.transforms = ColorHintTransform(self.size, mode)
    if mode == "training":
      train_dir = os.path.join(self.root_path, "train")
      self.examples = [os.path.join(self.root_path, "train", dirs) for dirs in os.listdir(train_dir)]
    elif mode == "validation":
      val_dir = os.path.join(self.root_path, "validation")
      self.examples = [os.path.join(self.root_path, "validation", dirs) for dirs in os.listdir(val_dir)]
    elif mode == "testing":
      hint_dir = os.path.join(self.root_path, "hint")
      mask_dir = os.path.join(self.root_path, "mask")
      self.hint = [os.path.join(self.root_path, "hint", dirs) for dirs in os.listdir(hint_dir)]
      self.mask = [os.path.join(self.root_path, "mask", dirs) for dirs in os.listdir(mask_dir)]
    else:
      raise NotImplementedError

  def __len__(self):
    if self.mode != "testing":
      return len(self.examples)
    else:
      return len(self.hint)

  def __getitem__(self, idx):
    if self.mode == "testing":
      hint_file_name = self.hint[idx]
      mask_file_name = self.mask[idx]
      hint_img = cv2.imread(hint_file_name)
      mask_img = cv2.imread(mask_file_name)

      input_l, input_hint = self.transforms(hint_img, mask_img)
      sample = {"l": input_l, "hint": input_hint,
                "file_name": "image_%06d.png" % int(os.path.basename(hint_file_name).split('.')[0])}
    else:
      file_name = self.examples[idx]
      img = cv2.imread(file_name)
      l, ab, hint = self.transforms(img)
      sample = {"l": l, "ab": ab, "hint": hint}

    return sample

# Network
Unet network for example.

In [10]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    # (convolution => [BN] => ReLU) * 2

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    # Downscaling with maxpool then double conv

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    # Upscaling then double conv

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose2d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [11]:
""" Full assembly of the parts to form the complete network """
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128)
        self.down2 = Down(128, 256)
        self.down3 = Down(256, 512)
        factor = 2 if bilinear else 1
        self.down4 = Down(512, 1024 // factor)
        self.up1 = Up(1024, 512 // factor, bilinear)
        self.up2 = Up(512, 256 // factor, bilinear)
        self.up3 = Up(256, 128 // factor, bilinear)
        self.up4 = Up(128, 64, bilinear)
        self.outc = OutConv(64, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        x = self.outc(x)
        return x

# Tensorboard
For training progress visualization. Run before training phase.

In [8]:
!pip install tensorboardX

     |████████████████████████████████| 122kB 9.5MB/s 


In [18]:
# %load_ext tensorboard
%reload_ext tensorboard

In [ ]:
%tensorboard --logdir logs

# Training Phase
Unet training code.

In [20]:
import torch
import torch.nn as nn
import torch.utils.data  as data

from torchvision import transforms
from tensorboardX import SummaryWriter
import torchvision.utils as tvutils

import os
import matplotlib.pyplot as plt
import numpy as np
import tqdm.notebook as tq
from PIL import Image
from skimage.measure.simple_metrics import compare_psnr

def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += compare_psnr(Iclean[i, :, :, :], Img[i, :, :, :], data_range=data_range)
    return (PSNR/Img.shape[0])

# Change to your data root directory
root_path = "/content/"
save_path = "/content/drive/MyDrive/Colorization_models"

# Depend on runtime setting
use_cuda = True

# Dataloader setting
train_dataset = ColorHintDataset(root_path, 128)
train_dataset.set_mode("training")

val_dataset = ColorHintDataset(root_path, 128)
val_dataset.set_mode("validation")

train_dataloader = data.DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = data.DataLoader(val_dataset, batch_size=4, shuffle=True)

# Model declaration
net = UNet(2, 2)
model = nn.DataParallel(net)

# loss
criterion = nn.MSELoss(size_average=False)

if use_cuda:
  model.to('cuda')
  criterion.to('cuda')

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, eps=1e-08)

step = 0
# Tensorboard writer
writer = SummaryWriter("logs")

for epoch in range(30):
  print('Epoch {}/{}'.format(epoch + 1, 30))
  print('-' * 10)

  for i, data in enumerate(tq.tqdm(train_dataloader)):
    if use_cuda:
      l = data["l"].to('cuda')
      ab = data["ab"].to('cuda')
      hint = data["hint"].to('cuda')
    
    model.train()
    model.zero_grad()
    optimizer.zero_grad()

    preds = model(hint)

    loss = criterion(preds, ab)

    loss.backward()
    optimizer.step()
    
    gt_image = torch.cat((l, ab), dim=1)
    pred_image = torch.cat((l, preds), dim=1)
    psnr_train = batch_PSNR(gt_image, pred_image, 1.)

    if step % 100 == 0:
      # Log the scalar values
      writer.add_scalar('loss', loss.item(), step)
      writer.add_scalar('PSNR on training data', psnr_train, step)

      # log the images => Tensorboard
      Img = tvutils.make_grid(gt_image.data, nrow=4, normalize=True, scale_each=True)
      Irecon = tvutils.make_grid(pred_image.data, nrow=4, normalize=True, scale_each=True)
      writer.add_image('GT image', Img, epoch)
      writer.add_image('reconstructed image', Irecon, epoch)
      print("[epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f" %
          (epoch + 1, i + 1, len(train_dataloader), loss.item(), psnr_train))
    step += 1
    
  torch.save(model.module.state_dict(), os.path.join(save_path, "{}.tar".format(epoch+1)))
  print("saved at {}".format(os.path.join(save_path, "{}.tar".format(epoch+1))))

  psnr_val = []

  # Validation on training phase
  model.eval()
  with torch.no_grad():
    for val_data in tq.tqdm(val_dataloader):
      l = val_data["l"].to('cuda')
      ab = val_data["ab"].to('cuda')
      hint = val_data["hint"].to('cuda')

      preds = model(hint)

      gt_image = torch.cat((l, ab), dim=1)
      pred_image = torch.cat((l, preds), dim=1)
      psnr = batch_PSNR(gt_image, pred_image, 1.)
      psnr_val.append(psnr)

      val_Img = tvutils.make_grid(gt_image.data, nrow=4, normalize=True, scale_each=True)
      val_Irecon = tvutils.make_grid(pred_image.data, nrow=4, normalize=True, scale_each=True)
      writer.add_image('validation gt image', val_Img, epoch)
      writer.add_image('validation reconstructed image', val_Irecon, epoch+1)
    
    mean_val = np.mean(psnr_val)
    print("\n[epoch %d] PSNR_val: %.4f" % (epoch + 1, mean_val))
    writer.add_scalar('PSNR on validation data', mean_val, epoch)

Epoch 1/30
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: UserWarning: DEPRECATED: skimage.measure.compare_psnr has been moved to skimage.metrics.peak_signal_noise_ratio. It will be removed from skimage.measure in version 0.18.


[epoch 1][1/1125] loss: 34781.6680 PSNR_train: 7.5238
[epoch 1][101/1125] loss: 491.2900 PSNR_train: 26.7685
[epoch 1][201/1125] loss: 813.3784 PSNR_train: 24.4462
[epoch 1][301/1125] loss: 177.8844 PSNR_train: 31.3786
[epoch 1][401/1125] loss: 241.8098 PSNR_train: 30.8565
[epoch 1][501/1125] loss: 305.2758 PSNR_train: 28.4273
[epoch 1][601/1125] loss: 245.5213 PSNR_train: 29.0979
[epoch 1][701/1125] loss: 182.4152 PSNR_train: 30.7988
[epoch 1][801/1125] loss: 103.3732 PSNR_train: 33.3172
[epoch 1][901/1125] loss: 466.1079 PSNR_train: 26.8440
[epoch 1][1001/1125] loss: 138.0480 PSNR_train: 31.6171
[epoch 1][1101/1125] loss: 182.2884 PSNR_train: 31.9554

saved at /content/drive/MyDrive/Colorization_models/1.tar




[epoch 1] PSNR_val: 32.0265
Epoch 2/30
----------


...



[epoch 28] PSNR_val: 34.8886
Epoch 29/30
----------


[epoch 29][1/1125] loss: 87.0516 PSNR_train: 34.2934
[epoch 29][101/1125] loss: 111.3662 PSNR_train: 32.7065
[epoch 29][201/1125] loss: 80.0166 PSNR_train: 35.0016
[epoch 29][301/1125] loss: 73.5902 PSNR_train: 35.3129
[epoch 29][401/1125] loss: 67.4658 PSNR_train: 35.3156
[epoch 29][501/1125] loss: 55.4081 PSNR_train: 37.3551
[epoch 29][601/1125] loss: 56.3381 PSNR_train: 35.8394
[epoch 29][701/1125] loss: 64.8414 PSNR_train: 35.6501
[epoch 29][801/1125] loss: 35.9042 PSNR_train: 38.2839
[epoch 29][901/1125] loss: 68.5382 PSNR_train: 37.5245
[epoch 29][1001/1125] loss: 84.2052 PSNR_train: 33.7745
[epoch 29][1101/1125] loss: 85.8015 PSNR_train: 34.3889

saved at /content/drive/MyDrive/Colorization_models/29.tar




[epoch 29] PSNR_val: 31.2557
Epoch 30/30
----------


[epoch 30][76/1125] loss: 119.5296 PSNR_train: 33.6934
[epoch 30][176/1125] loss: 133.9970 PSNR_train: 34.2125
[epoch 30][276/1125] loss: 46.9104 PSNR_train: 36.6876
[epoch 30][376/1125] loss: 97.0827 PSNR_train: 33.9296
[epoch 30][476/1125] loss: 45.4628 PSNR_train: 36.5721
[epoch 30][576/1125] loss: 50.6068 PSNR_train: 37.3620
[epoch 30][676/1125] loss: 60.7454 PSNR_train: 36.4470
[epoch 30][776/1125] loss: 44.4611 PSNR_train: 36.7821
[epoch 30][876/1125] loss: 43.2207 PSNR_train: 37.7906
[epoch 30][976/1125] loss: 74.1865 PSNR_train: 34.4992
[epoch 30][1076/1125] loss: 46.5415 PSNR_train: 36.5116

saved at /content/drive/MyDrive/Colorization_models/30.tar




[epoch 30] PSNR_val: 31.2059


In [17]:
import torch
import torch.nn as nn
import torch.utils.data  as data

import os
import cv2
import numpy as np
import tqdm.notebook as tq
from PIL import Image
from skimage.measure.simple_metrics import compare_psnr

def image_save(img, path):
  if isinstance(img, torch.Tensor):
    img = np.asarray(transforms.ToPILImage()(img))
  img = cv2.cvtColor(img, cv2.COLOR_LAB2BGR)
  cv2.imwrite(path, img)

def batch_PSNR(img, imclean, data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR = 0
    for i in range(Img.shape[0]):
        PSNR += compare_psnr(Iclean[i, :, :, :], Img[i, :, :, :], data_range=data_range)
    return (PSNR/Img.shape[0])

# Change to your data root directory
image_path = "/content/drive/MyDrive/Multimedia_test_dataset/colorization2/"
checkpoint_path = "/content/drive/MyDrive/Colorization_models/25.tar"
result_save_path = "/content/drive/MyDrive/Multimedia_test_dataset/colorization_test_result"

# Depend on runtime setting
use_cuda = True

test_dataset = ColorHintDataset(image_path, 128)
test_dataset.set_mode("testing")

test_dataloader = data.DataLoader(test_dataset, batch_size=1, shuffle=False)

net = UNet(2, 2)

if use_cuda:
  net.to('cuda')

net.load_state_dict(torch.load(checkpoint_path))
model = nn.DataParallel(net)

model.eval()

for i, data in enumerate(tq.tqdm(test_dataloader)):
  if use_cuda:
    l = data["l"].to('cuda')
    hint = data["hint"].to('cuda')
  file_name = data["file_name"]

  with torch.no_grad():
    out_test = model(hint)
    pred_image = torch.cat((l, out_test), dim=1)
    for idx in range(len(file_name)):
      image_save(pred_image[idx], os.path.join(result_save_path, file_name[idx]))
